## 1. Chargement et Préparation des Données

### 1.1 Importation des données

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


### 1.2 Load and Merge Data (Charger et Fusionner les Données)
Objectif : Charger tous les fichiers CSV, puis fusionner les données sur EmployeeID.

In [49]:

employee_df = pd.read_csv("data/employee_survey_data.csv")
general_df = pd.read_csv("data/general_data.csv")
manager_df = pd.read_csv("data/manager_survey_data.csv")
in_time_df = pd.read_csv("data/in_time.csv")
out_time_df = pd.read_csv("data/out_time.csv")

# Merge employee_df and general_df on EmployeeID
merged_df = pd.merge(employee_df, general_df, on="EmployeeID", how="inner")

# Merge in manager_df
merged_df = pd.merge(merged_df, manager_df, on="EmployeeID", how="inner")


# Afficher un aperçu
print(merged_df.head())


   EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  Age  \
0           1                      3.0              4.0              2.0   51   
1           2                      3.0              2.0              4.0   31   
2           3                      2.0              2.0              1.0   32   
3           4                      4.0              4.0              3.0   38   
4           5                      4.0              1.0              3.0   32   

  Attrition     BusinessTravel              Department  DistanceFromHome  \
0        No      Travel_Rarely                   Sales                 6   
1       Yes  Travel_Frequently  Research & Development                10   
2        No  Travel_Frequently  Research & Development                17   
3        No         Non-Travel  Research & Development                 2   
4        No      Travel_Rarely  Research & Development                10   

   Education  ... PercentSalaryHike  StandardHours Stock

### 1.3 Drop Irrelevant Columns (Supprimer les Colonnes Inutiles)
Objectif : Supprimer les colonnes qui n'apportent pas d'informations utiles à l'analyse.
#### Colonnes à supprimer :
- Over18 : Tous les employés ont plus de 18 ans.
- EmployeeCount : Toujours égal à 1 (inutile).
- StandardHours : Toujours égal à 8 (inutile).

In [50]:
# Supprimer les colonnes inutiles
columns_to_drop = ["EmployeeCount", "Over18", "StandardHours"]
merged_df.drop(columns=columns_to_drop, axis=1, inplace=True, errors="ignore")
# Visualiser apres suppression de quelques colonnes
print(merged_df.head())


   EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  Age  \
0           1                      3.0              4.0              2.0   51   
1           2                      3.0              2.0              4.0   31   
2           3                      2.0              2.0              1.0   32   
3           4                      4.0              4.0              3.0   38   
4           5                      4.0              1.0              3.0   32   

  Attrition     BusinessTravel              Department  DistanceFromHome  \
0        No      Travel_Rarely                   Sales                 6   
1       Yes  Travel_Frequently  Research & Development                10   
2        No  Travel_Frequently  Research & Development                17   
3        No         Non-Travel  Research & Development                 2   
4        No      Travel_Rarely  Research & Development                10   

   Education  ... NumCompaniesWorked PercentSalaryHike  

### 1.4 Clean up and Prepare Time Data (Nettoyage et Préparation des Données de Temps)
Objectif : Transformer les fichiers in_time et out_time pour obtenir le nombre d'heures travaillées.

#### Actions :
- Convertir les dates en format datetime.
- Calculer la durée travaillée chaque jour (out_time - in_time).
- Calculer la moyenne des heures travaillées pour chaque employé.

In [51]:
# Rename 'Unnamed: 0' to 'EmployeeID' for easier merging
in_time_df.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
out_time_df.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)

# We’ll create a separate DataFrame that holds each employee’s average work hours
# Convert time columns to datetime (ignore errors if any NaNs or incorrect formats appear)
time_cols = in_time_df.columns[1:]  # skip EmployeeID column
in_time_df[time_cols] = in_time_df[time_cols].apply(pd.to_datetime, errors="coerce")
out_time_df[time_cols] = out_time_df[time_cols].apply(pd.to_datetime, errors="coerce")

# Calculate the daily hours as OUT - IN
# This returns a DataFrame of timedeltas; convert to hours by dividing total_seconds by 3600
daily_hours = out_time_df[time_cols].sub(in_time_df[time_cols])
daily_hours = daily_hours.apply(lambda row: row.dt.total_seconds() / 3600)

# Create average hours feature per employee
avg_hours_df = pd.DataFrame({
    "EmployeeID": in_time_df["EmployeeID"],
    "AverageWorkHours": daily_hours.mean(axis=1)  # mean across all days
})

# Merge avg_hours_df with merged_df
merged_df = pd.merge(merged_df, avg_hours_df, on="EmployeeID", how="left")

print(merged_df[["EmployeeID", "AverageWorkHours"]].head())


   EmployeeID  AverageWorkHours
0           1          7.373651
1           2          7.718969
2           3          7.013240
3           4          7.193678
4           5          8.006175


### 1.5 Merge Time Features Back (Fusionner les Données de Temps dans le Dataset Principal)
**Objectif** : Ajouter la colonne AverageWorkHours au dataset fusionné.

In [52]:
# Fusionner les données de temps avec le dataset principal
final_df = pd.merge(merged_df, avg_hours_df, on="EmployeeID", how="left")

# Vérifier les mises à jour
print(final_df.head())


   EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  Age  \
0           1                      3.0              4.0              2.0   51   
1           2                      3.0              2.0              4.0   31   
2           3                      2.0              2.0              1.0   32   
3           4                      4.0              4.0              3.0   38   
4           5                      4.0              1.0              3.0   32   

  Attrition     BusinessTravel              Department  DistanceFromHome  \
0        No      Travel_Rarely                   Sales                 6   
1       Yes  Travel_Frequently  Research & Development                10   
2        No  Travel_Frequently  Research & Development                17   
3        No         Non-Travel  Research & Development                 2   
4        No      Travel_Rarely  Research & Development                10   

   Education  ... StockOptionLevel TotalWorkingYears  Tr

### 1.6 Create or Transform Other Meaningful Features (Créer ou Transformer des Variables Intéressantes)
**Objectif** : Transformer les variables en formats exploitables pour l’analyse et les modèles.

#### Transformations importantes :
Encodage de Attrition : Convertir Yes/No en 1/0.
Remplacer les valeurs manquantes (NA) dans employee_survey_data.csv par la médiane.
Encodage des variables catégorielles (BusinessTravel, EducationField, etc.).
Création d'une variable YearsSinceLastPromotionRatio : YearsSinceLastPromotion / YearsAtCompany.

In [53]:
# Transformer la colonne 'Attrition' en variable binaire
final_df["Attrition"] = final_df["Attrition"].map({"No": 0, "Yes": 1})

# Remplacer les valeurs manquantes par la médiane dans employee_survey_data
for col in ["EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance"]:
    final_df[col].fillna(final_df[col].median(), inplace=True)

# Encodage des variables catégorielles avec pandas get_dummies
final_df = pd.get_dummies(final_df, columns=["BusinessTravel", "EducationField", "Gender", "MaritalStatus", "JobRole"], drop_first=True)

# Créer une variable 'YearsSinceLastPromotionRatio'
final_df["YearsSinceLastPromotionRatio"] = final_df["YearsSinceLastPromotion"] / (final_df["YearsAtCompany"] + 1)

# Vérifier les mises à jour
print(final_df.head())


   EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  Age  \
0           1                      3.0              4.0              2.0   51   
1           2                      3.0              2.0              4.0   31   
2           3                      2.0              2.0              1.0   32   
3           4                      4.0              4.0              3.0   38   
4           5                      4.0              1.0              3.0   32   

   Attrition              Department  DistanceFromHome  Education  JobLevel  \
0          0                   Sales                 6          2         1   
1          1  Research & Development                10          1         1   
2          0  Research & Development                17          4         4   
3          0  Research & Development                 2          5         3   
4          0  Research & Development                10          1         1   

   ...  MaritalStatus_Single  JobRole_

C:\Users\fallo\AppData\Local\Temp\ipykernel_22244\4004663908.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\fallo\AppData\Local\Temp\ipykernel_22244\4004663908.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

## Préparation des Données
### 1. Gestion des valeurs manquantes

In [55]:
# Vérifier les valeurs manquantes
df.isnull().sum()

# Imputer les valeurs manquantes pour les variables numériques avec la médiane
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

# Sélection des colonnes numériques

## à Compléter
df_num = df.select_dtypes(include=float)
imputer.fit(df_num)
## à Compléter

# Remplacement des valeurs manquantes
df_num_imputed = pd.DataFrame(imputer.transform(df_num), columns=df_num.columns)

# Vérifier s'il reste des valeurs manquantes
df_num_imputed.isnull().sum()

EnvironmentSatisfaction         0
JobSatisfaction                 0
WorkLifeBalance                 0
NumCompaniesWorked              0
TotalWorkingYears               0
AverageWorkHours_x              0
AverageWorkHours_y              0
YearsSinceLastPromotionRatio    0
dtype: int64

### 2. Encodage des variables catégorielles

In [56]:
# Sélection des colonnes catégorielles
df_cat = df.select_dtypes(include=[object])

# Encodage one-hot des variables catégorielles

## à Compléter
df_cat_encoded = pd.get_dummies(df_cat)
## à Compléter 

# Vérification du résultat de l'encodage
df_cat_encoded.head()

,Department_Human Resources,Department_Research & Development,Department_Sales
0,False,False,True
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False


### 3. Normalization des données

### 4. Assemblage des données préparées

### 5. Séparation des caractéristiques (X) et de la cible (y)

## Exploration et Visualisation des Données

### 1. Analyse des Statistiques Descriptives

Avant de plonger dans la visualisation des données, examinons les statistiques descriptives pour obtenir une première idée des distributions des variables.

### 2. Tests d'ANOVA & X2
write something

### 3. Visualisation des Variables Numériques
Nous commencerons par visualiser les distributions des variables numériques pour mieux comprendre leurs comportements.

### 3. Matrice de Corrélation
Pour explorer les relations entre les variables numériques, nous utilisons une matrice de corrélation et une heatmap.

### 4. Visualisation des Relations entre les Variables
Nous allons créer quelques graphes de dispersion pour visualiser les relations entre certaines variables clés.

### 5. Analyse de la Variable Cible
Analysons la variable cible Sold6M pour voir comment elle est distribuée.

### 6. Relations entre les Variables et la Cible
Examinons comment certaines variables influencent la probabilité de vendre un bien immobilier dans les 6 mois.

## Modèles de Classification
Dans cette section, nous allons créer et évaluer plusieurs modèles de classification afin de prédire... (ici on met des definitions rapide et insights sur les modeles (interpretabilité vs performance...))

## Application des modèles
### 0. Préparation des données d'apprentissage et de test

### 1. Régerssion Logistique

Ici, ajouter plusieurs cellules (entrainement, evaluation, optimisation, explicabilité)

### 2. Random Forest

Ici, ajouter plusieurs cellules (entrainement, evaluation, optimisation, explicabilité)

# Etude comparative entre les modèles

## Interprétation des Résultats
### 1. Analyse des Performances

## Conclusion
### 1. Synthèse des Travaux Réalisés